In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from pyod.models.iforest import IForest
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import OneSidedSelection
import os
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.pca import PCA
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import GridSearchCV

import os, sys, pickle
import argparse, sys

import seaborn as sns
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')



# parser = argparse.ArgumentParser()
# parser.add_argument('-label', help=' : Please set the label') 
# args = parser.parse_args()


df = pd.read_csv('data/modified_dataset.csv')


# def remove_outliers(df, column_name, factor=1.5):
#     Q1 = df[column_name].quantile(0.25)
#     Q3 = df[column_name].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - factor * IQR
#     upper_bound = Q3 + factor * IQR
#     df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
#     return df_filtered

# # 모든 칼럼에 대해 이상치 제거 실행
# for column_name in df.columns:
#     df = remove_outliers(df, column_name)







df_stroke_0 = df[df['Cancer'] == 0]
df_stroke_1 = df[df['Cancer'] == 1]



#train, test data processing 1,2,3,4중 선택

#1.정상으로만 학습, 테스트는 정상과 이상 반반
answer_label = 'Cancer'
X = df[df.columns.difference([answer_label])]
df_normal = df[df[answer_label] == 0]
df_abnormal = df[df[answer_label] == 1]
test_normal_df = df_normal.sample(n=len(df_stroke_1), random_state = 0)
test_df = pd.concat([df_abnormal, test_normal_df])
X_test = test_df[test_df.columns.difference([answer_label])]
y_test = test_df[answer_label]
train_df = df_normal.drop(test_normal_df.index)
X_train = train_df[train_df.columns.difference([answer_label])]
y_train = train_df[answer_label]



In [11]:

def main_task(n_components,whiten):
    pca = PCA(n_components=n_components, whiten=whiten)
    pca.fit(X_train)
    PCA_test_pred = pca.predict(X_test)
    return round(f1_score(y_test, PCA_test_pred),3)

In [12]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

def optuna_main(trial: optuna.Trial):
    n_components = trial.suggest_int('n_components', 1, 5, step = 1)
    # svd_solver = trial.suggest_categorical('svd_solver', [ "auto", "full", "arpack", "randomized" ])
    whiten = trial.suggest_categorical('whiten':[True, False])
    print('n_components : ', n_components)
    # print('svd_solver : ', svd_solver)
    print('whiten : ', whiten)    
    f1_score = main_task(n_components=n_components, whiten=whiten)

    print('<f1_score:', f1_score,'>')
    return f1_score

In [13]:
study = optuna.create_study(direction='maximize')
study.optimize(optuna_main, n_trials=100)
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_params : {study.best_trial.params}')

[I 2023-11-08 14:30:24,772] A new study created in memory with name: no-name-13bd932c-29b9-4fcd-ac93-70a71494973b
[I 2023-11-08 14:30:24,847] Trial 0 finished with value: 0.755 and parameters: {'n_components': 1, 'svd_solver': 'arpack'}. Best is trial 0 with value: 0.755.
[I 2023-11-08 14:30:24,935] Trial 1 finished with value: 0.779 and parameters: {'n_components': 2, 'svd_solver': 'auto'}. Best is trial 1 with value: 0.779.
[I 2023-11-08 14:30:24,967] Trial 2 finished with value: 0.779 and parameters: {'n_components': 2, 'svd_solver': 'auto'}. Best is trial 1 with value: 0.779.
[W 2023-11-08 14:30:24,977] Trial 3 failed with parameters: {'n_components': 5, 'svd_solver': 'arpack'} because of the following error: ValueError("n_components=5 must be strictly less than min(n_samples, n_features)=5 with svd_solver='arpack'").
Traceback (most recent call last):
  File "/home/mohee/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = f

n_components :  1
svd_solver :  arpack
<f1_score: 0.755 >
n_components :  2
svd_solver :  auto
<f1_score: 0.779 >
n_components :  2
svd_solver :  auto
<f1_score: 0.779 >
n_components :  5
svd_solver :  arpack


ValueError: n_components=5 must be strictly less than min(n_samples, n_features)=5 with svd_solver='arpack'

In [7]:
#결과
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_paramse : {study.best_params}')

study.best_trial.value : 0.799
study.best_paramse : {'n_components': 3}
